# Burocracy
1. checking GPU
2. install dependencies
3. import libraries

In [1]:
##1. setting gpu
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found GPU at: /device:GPU:0


In [ ]:
## 2. installation: some of these are requiered for pacman environment

!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null

!apt-get install x11-utils > /dev/null 2>&1
!pip install pyglet > /dev/null 2>&1

In [ ]:
#installing gym things for atari. Guess u need this version here
!pip install -U gym>=0.21.0
!pip install -U gym[atari,accept-rom-license]

In [ ]:
import gym
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
#display plot in a notebook
%matplotlib inline
import random
from collections import deque, Counter
from datetime import datetime

# ML libraries
import tensorflow as tf
fimport matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten

#graphic render of the environment
from IPython import display as ipythondisplay

# Pacman environment
the goal is to teach the computer to play Pacman. Let's look at the environment

In [ ]:
#needed but don't know why, guess it's something with ale. Still, this way it works
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import MsPacman
ale.loadROM(MsPacman)

In [ ]:
env = gym.make('ALE/MsPacman-v5', render_mode="rgb_array")
## useful if u want to look at the environment
#prev_screen = env.render()
#plt.imshow(prev_screen)
#ipythondisplay.clear_output(wait=True)

# Define functions i need

## Process image
Now we define a function called preprocess_observation for preprocessing our input game screen. We reduce the image size and convert the image into greyscale.



In [ ]:
#functions for preprocessing

def cropping(start_obs):
  # Cropping image: to choose the value, u should look at the image
  img = start_obs[1:176:2,::2]
  #show modified image
  '''
  print('Cropping image')
  plt.imshow(img)
  ipythondisplay.clear_output(wait=True)
  '''
  return img

def to_greyscale(img):
  # Convert the image to greyscale
  img = img.mean(axis=2)
  #show modified image
  '''
  print('Turn to greyscale')
  plt.imshow(img)
  ipythondisplay.clear_output(wait=True)
  '''
  return img

def improve_contrast(img):
  # Improve image contrast
  color = np.array([210, 164, 74]).mean()
  img[img==color] = 0
  #show modified image
  '''
  print('More contrast')
  plt.imshow(img)
  ipythondisplay.clear_output(wait=True)
  '''
  return img

def normalize(img):
  #Normalize the image from -1 to +1
  img = (img - 128) / 128 - 1
  #show modified image
  '''
  print('Normalize')
  plt.imshow(img)
  ipythondisplay.clear_output(wait=True)
  '''
  return img

**REMEMBER: gotta pass to preprocess_observation, observation[0], and not all the observation matrix**

In [ ]:
def preprocess_observation(obs):
  #1. crop
  image = cropping(obs)
  #2. greyscale
  image = to_greyscale(image)
  #3. contrast
  image = improve_contrast(image)
  #4. normalize in [-1;+1]
  image = normalize(image)
  # 5. reshape image
  image = image.reshape(88,80,1)
  return image

## Building Q network
Q network is the predicting network, which is used to predict the weights. The input is the game state X. The output has the dimension of the action space, so it's one place for one action.

The Q network below has three convolutional layers with SAME padding, followed by a fully connected layer.

In [ ]:
def q_network(screen_shape, learning_rate):
    #input
    start_state = Input(shape = screen_shape)

    #initialize weights
    w_start = tf.keras.initializers.VarianceScaling()

    #layers
    x = Conv2D(32, kernel_size = (8,8), strides = 4, padding = 'SAME',\
               kernel_initializer =w_start, name = 'conv_1' )(start_state) #first layer
    x = Conv2D(64, kernel_size = (4,4), strides = 2, padding = 'SAME',\
               kernel_initializer =w_start, name = 'conv_2' )(x) #second layer
    x = Conv2D(64, kernel_size = (3,3), strides = 1, padding = 'SAME',\
               kernel_initializer =w_start, name = 'conv_3' )(x) #thrid layer
    flatten = Flatten(name='flatten')(x) #flatten output
    fully_connected = Dense(128, kernel_initializer =w_start, name = 'fc')(flatten)  #fully connected layer
    out_layer = Dense(18,kernel_initializer =w_start, name = 'output')(fully_connected) #output

    model = Model(start_state, out_layer)

    #summarize and compile model
    model.summary()
    model.compile(optimizer= Adam(lr = learning_rate), loss = "mse")

    return model

**NOTE**: the name 'SAME' padding just came from the property that when stride equals 1, output spatial shape is the same as input spatial shape. In general case, the definition of 'SAME' means to apply the padding in a tensorflow way such that:

*For each spatial dimension i:
output_spatial_shape[i] = ceil(input_spatial_shape[i] / strides[i])*.


Basically, padding = 'SAME' concers the dimention of input and output.

## Epsilon greedy policy
The value of epsilon decay over time

In [ ]:
def epsilon_greedy(action, step,eps_min,eps_max,eps_decay_steps,n_outputs):
    tmp = eps_max - (eps_max-eps_min) * step/eps_decay_steps
    epsilon = max(eps_min,tmp)
    rand_tmp = np.random.rand()
    if rand_tmp < epsilon:
        return np.random.randint(n_outputs) #returna un numero tra 0 e n output
    else:
        return action

## Replay buffer

In [ ]:
def sample_memories(batch_size, exp_buffer):
    perm_batch = np.random.permutation(len(exp_buffer))[:batch_size]
    # permuta in modo randomico np.arange(x), e poi prendiamo solo la prima botta di batch
    print('perm_batch')
    print(perm_batch)
    mem = np.array(exp_buffer)[perm_batch]
    print('mem')
    print(mem)
    return [mem[:,0], mem[:,1], mem[:,2], mem[:,3], mem[:,4]]



# Main

## Hyperparameters
Parameters whose value is used to control learning *process*

In [ ]:
n_action = env.action_space.n
n_episode = 800

# nn variables
batch_size = 48
input_shape = (88,80,1)
learning_rate_alpha = 0.001

#epsilon greedy policy variables
min_eps = 0.05
max_eps = 1
decay_steps_epsilon = 500000 

# training variables
discount_factor = 0.97
global_step = 0
steps_train = 4
start_steps = 2000
copy_steps = 100

In [ ]:
# Build Q network, that takes input X, initial screen state and generates Q values for all the actions
#in the state
Q_network = q_network(input_shape, learning_rate_alpha)
target_Q_network = q_network(input_shape, learning_rate_alpha)